# 2DEG mobility calculation using analytical formula

Ref-1: J. Bassaler, J. Mehta, I. Abid, L. Konczewicz, S. Juillaguet, S. Contreras, S. Rennesson, S. Tamariz, M. Nemoz, F. Semond, J. Pernot, F. Medjdoub, Y. Cordier, P. Ferrandis, Al-Rich AlGaN Channel High Electron Mobility Transistors on Silicon: A Relevant Approach for High Temperature Stability of Electron Mobility. Adv. Electron. Mater. 2024, 2400069. https://doi.org/10.1002/aelm.202400069

Ref-2: Zhang, J., Hao, Y., Zhang, J. et al. The mobility of two-dimensional electron gas in AlGaN/GaN heterostructures with varied Al content. Sci. China Ser. F-Inf. Sci. 51, 780–789 (2008). https://doi.org/10.1007/s11432-008-0056-7


# 1. Settings

## 1.1 Import modules

In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
from pathlib import Path
mobilitypy_tutorial_path = Path().absolute()
sys.path.insert(0, os.path.join(mobilitypy_tutorial_path, '../'))
from mobilitypy import AlloyParams, Mobility2DCarrier, Plottings

In [4]:
save_figure_dir = os.path.join(mobilitypy_tutorial_path,'../imgs')
save_file_name = 'mobilities'
savefigure = True
fig_dpi = 75

## 1.2 Set physical constants

In [5]:
'''
n_2d => in nm^-2
rms_roughness => nm^-1
corr_len => nm^-1
n_dis => nm^-2
f_dis => unit less
'''
alloy_disordered_effect=1
interface_roughness_effect=1
dislocation_effect=1
deformation_potential_effect=1 
piezoelectric_effect=1
acoustic_phonon_effect=1
polar_optical_phonon_effect=1
total_mobility=1
mobility_model='Bassaler'
density_2deg = 0.1 # nm^-2
irf_rms_roughness = 0.3 # nm
irf_corr_length = 3.0 # nm
dislocation_density = 1e-4 # nm^-2
occup_dislocation = 0.3
T=300 #K

# 2. Calculate mobilities

In [6]:
compositions = np.linspace(0,1,101)
mu2deg = Mobility2DCarrier(compositions=compositions, binaries=['AlN', 'GaN'], alloy='AlGaN', system='ternary')

In [7]:
mobility_df = mu2deg.calculate_sheet_mobility(n_2d=density_2deg, rms_roughness=irf_rms_roughness, 
                                              corr_len=irf_corr_length, n_dis=dislocation_density, 
                                              f_dis=occup_dislocation, T=T,
                                              alloy_disordered_effect=alloy_disordered_effect,
                                              interface_roughness_effect=interface_roughness_effect,
                                              dislocation_effect=dislocation_effect,
                                              deformation_potential_effect=deformation_potential_effect,
                                              piezoelectric_effect=piezoelectric_effect,
                                              acoustic_phonon_effect=acoustic_phonon_effect,
                                              polar_optical_phonon_effect=polar_optical_phonon_effect,
                                              total_mobility=total_mobility,
                                              mobility_model=mobility_model)

In [8]:
mobility_df

,comp,IFR,AD,DIS,DP,PE,AP,POP,TOT
0,0.000,27877.662014,NaN,50387.952966,9081.733838,19864.282032,6232.364520,1987.595360,1390.259284
1,0.010,27683.710698,17031.262347,50145.183770,8986.905363,19385.552960,6140.325518,1975.475576,1275.762412
2,0.020,27492.351320,8132.195863,49904.809583,8893.175338,18922.899750,6049.906928,1963.491834,1171.003431
3,0.030,27303.535928,5186.509676,49666.794591,8800.524909,18475.621722,5961.075491,1951.641863,1075.602583
4,0.040,27117.217699,3727.587558,49431.103686,8708.935640,18043.056110,5873.798702,1939.923441,989.013075
...,...,...,...,...,...,...,...,...,...
96,0.960,16674.391963,599.810344,34501.856331,3217.143663,3745.479734,1730.633082,1228.916196,317.692468
97,0.970,16606.277172,791.847607,34389.498289,3177.999721,3697.827059,1709.131678,1223.782482,363.055844
98,0.980,16538.766704,1176.643576,34277.870689,3139.138835,3651.009205,1687.890266,1218.684899,425.075257
99,0.990,16471.853104,2332.442970,34166.966092,3100.557696,3605.007675,1666.904082,1213.623037,514.772341


In [9]:
composition_axis = np.array(mobility_df['comp'], dtype=float)
mu_total = np.array(mobility_df['TOT'], dtype=float)

In [10]:
# Unit of sheet resitance: ohm per square
sheet_resistance = mu2deg.calculate_sheet_resitance(density_2deg, mu_total)
data_sheet_resitance_2_plot = np.column_stack((composition_axis, sheet_resistance))

In [11]:
# Unit of lateral_FOM: MW/cm^2
lateral_FOM = mu2deg.calculate_figure_of_merit(density_2deg, mu_total)
data_FOM_2_plot = np.column_stack((composition_axis, lateral_FOM))# LFOM: MW/cm^2

# 3. Plot mobilities

In [12]:
plt2deg = Plottings(save_figure_dir=save_figure_dir)
x_label = 'Al composition'
y_label = r'Electron mobility ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)'

In [13]:
save_file_name_ = f'{save_file_name}_{T}K.png'
fig, ax,_ = plt2deg.plot_2d_carrier_mobilities(mobility_df, save_file_name=save_file_name_,
                                               ymin=None, ymax=None, xmax=1, xmin=0, y_scale_log=True, 
                                               annotate_pos=(11,11), show_right_ticks=True,
                                               mode='2d_carrier_mobility', yaxis_label=y_label, 
                                               xaxis_label=x_label, color=None, color_map='viridis', 
                                               savefig=savefigure, dpi=fig_dpi, show_plot=True)

In [14]:
save_file_name_ = f'LFOM_{T}K.png'
fig, ax, _ = plt2deg.plot_2d(data_FOM_2_plot, save_file_name=save_file_name_,
                             ymin=None, ymax=None, xmax=1.0, xmin=0, y_scale_log=True,
                             yaxis_label=r'LFOM (MW/cm$^2$)', xaxis_label=x_label,
                             color='k', color_map='viridis', savefig=savefigure, 
                             dpi=fig_dpi, show_plot=False)
ax.axhline(y=data_FOM_2_plot[0,1], c='k', ls='--')
#plt.show()

In [15]:
save_file_name_ = f'LFOM_norm_{T}K.png'
YY = data_FOM_2_plot.copy()
YY[:, 1] /= YY[0, 1]
fig, ax, _ = plt2deg.plot_2d(YY, save_file_name=save_file_name_,
                             ymin=None, ymax=None, xmax=1, xmin=0, y_scale_log=True,
                             yaxis_label=r'LFOM$_\mathrm{AlGaN}$ / LFOM$_\mathrm{GaN}$', 
                             xaxis_label=x_label, color='k', color_map='viridis', 
                             savefig=0, show_plot=False)
ax.axhline(y=1, c='k', ls='--')
plt2deg.save_figure(save_file_name_,fig=fig, savefig=savefigure, dpi=fig_dpi)

In [17]:
save_file_name_ = f'sheet_resistamce_{T}K.png'
#data_sheet_resitance_2_plot[:, 1] /= data_sheet_resitance_2_plot[0, 1]
fig, ax, _ = plt2deg.plot_2d(data_sheet_resitance_2_plot, save_file_name=save_file_name_,
                             ymin=None, ymax=None, xmax=1, xmin=0, y_scale_log=True,
                             yaxis_label=r'Sheet resistance ($\Omega /\boxdot$)', 
                             xaxis_label=x_label, color='k', color_map='viridis', 
                             savefig=0, show_plot=False)
#ax.axhline(y=data_FOM_2_plot[0,1], c='k', ls='--')
plt2deg.save_figure(save_file_name_,fig=fig, savefig=savefigure, dpi=fig_dpi)